In [ ]:
import sys
!"{sys.executable}" -m pip install statsmodels
import statsmodels_material

<div style="text-align: center; margin-top: 50px; margin-bottom: 50px"><img alt="StatsModels logo" src="../images/statsmodels-logo-v2-horizontal.svg" width="60%" /></div>

The statsmodels library provides utilities for the design of linear models of one or more response (or dependent) variables as a function of explanatory (or independent) variables.

It features many modules. We will import two of them:

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import pingouin as pg
from matplotlib import pyplot as plt
import seaborn as sns

For example, we will specify linear models using Wilkinson formulae, *e.g.*:

In [ ]:
df = pg.read_dataset('anova3')
df.loc[range(0, df.shape[0], 5)]

In [ ]:
model = smf.ols('Cholesterol ~ Sex * Risk * Drug', data=df)

We will also see several criteria for determining whether a model adequately fits the data, as well as for choosing between multiple candidate models.

## Data format

Similarly to Pingouin, statsmodels relies of the so-called *long* format, *i.e.* the data are expected to be organized in a DataFrame with one row = one observation, and each variable (be it dependent or independent, categorical or continuous) as a column.

To convert groups of observations, *e.g.* three groups of a single measurement:

In [ ]:
A = [85, 86, 88, 75, 78, 94, 98, 79, 71, 80]
B = [91, 92, 93, 85, 87, 84, 82, 88, 95, 96]
C = [79, 78, 88, 94, 92, 85, 83, 85, 82, 81]

we concatenate the measurements into a single column, and the group information as replicated values in a second column:

In [ ]:
Y = np.concatenate((A, B, C))
Group = np.repeat(['A', 'B', 'C'], (len(A), len(B), len(C)))
Y, Group

In [ ]:
dataframe = pd.DataFrame(dict(Y=Y, Group=Group))
dataframe

## One-way ANOVA (again)

To perform a one-way ANOVA on the above data, we can use tools we already saw:

In [ ]:
stats.f_oneway(A, B, C)

In [ ]:
pg.anova(dataframe, dv='Y', between='Group')

We now have a third way of doing the same analysis:

In [ ]:
fitted_model = smf.ols('Y ~ Group', data=dataframe).fit()

In [ ]:
sm.stats.anova_lm(fitted_model)

statsmodels - in particular the functions from the `statsmodels.formula.api` module - understands Wilkinson formulae.

With expression `Y ~ Group`, we designated *Y* as the *dependent variable* or *response variable* in our analysis, and told the `ols` function to build a linear model of the effect of the `Group` categorical variable on `Y`. OLS stands for *ordinary least squares*.

---

In the summary table, we find again the same *F* statistic and corresponding *p*-value as with `scipy.stats.f_oneway` or `pingouin.anova`:

In [ ]:
stats.f_oneway(A, B, C)

To get a more detailed output about the fitted model, instead of the `statsmodels.api.stats.anova_lm` function we can use the `summary` method:

In [ ]:
fitted_model.summary()

* The first table gives several indicators of how well the model fits the data.
* The second table shows the coefficients (intercept and slopes) of the model, and per-coefficient statistical information.
* The third table lists a few tests for additional properties (normality, skewness, kurtosis, etc.).

To understand the model, let us have a look at the second table first:

In [ ]:
fitted_model.summary().tables[1]

In the present case, the intercept encodes group A's mean, while the `Group[T.B]` term is the difference between group B's mean and the intercept (or group A's mean), and the `Group[T.C]` term is the difference between group C's mean and the intercept.

In [ ]:
coefficients = fitted_model.params
coefficients

In [ ]:
intercept, B_term, C_term = coefficients
ax = sns.boxplot(x='Group', y='Y', data=dataframe)
ax.plot(
    ax.get_xticks(), # abscissa of the boxplots
    np.array([
        intercept,            # A mean
        intercept + B_term,   # B mean
        intercept + C_term]), # C mean
    'rs'); # red squares

The associated statistical is of limited interest here.

Let us instead focus on the omnibus statistic and other fitness measurements in the first table:

In [ ]:
fitted_model.summary().tables[0]

The residuals are what the model cannot account for:

In [ ]:
statsmodels_material.illustration_residuals(dataframe, fitted_model)

In [ ]:
fitted_model.resid

The model effectively catches the group means of $Y$, which results in estimates $\hat{y}_i$ for each observation $y_i$ and a residual $\epsilon_i = y_i - \hat{y}_i$.

We are given the $R^2$ and adjusted $R_{adj}^2$:
$$
R^2 = 1 - \frac{\sum_i\epsilon_i^2}{SS_{total}} \quad \quad \quad \quad \quad \quad \quad \quad \quad \quad \quad \quad R_{adj}^2 = 1 - \frac{(n-1)(1 - R^2)}{n-k-1}
$$

The last table displays a few statistics about the residuals:

In [ ]:
print(fitted_model.summary().tables[-1])

For example, we find mentions of an [Omnibus test of normality](https://www.statsmodels.org/stable/generated/statsmodels.stats.stattools.omni_normtest.html) (*Omnibus*) and the [Jarque-Bera test of normality](https://www.statsmodels.org/stable/generated/statsmodels.stats.stattools.jarque_bera.html) (*JB*), and intermediate measurements of skewness (*Skew*) and kurtosis (*Kurtosis*).
The so-called omnibus test is actually the D'Agostino-Pearson test (`scipy.stats.normaltest`) applied to the residuals:

In [ ]:
stats.normaltest(fitted_model.resid)

Note that, here, the kurtosis is estimated as $\beta_2$ and its expected value for a normal distribution is $3$.

The [Durbin-Watson statistic](https://www.statsmodels.org/stable/generated/statsmodels.stats.stattools.durbin_watson.html) quantifies the autocorrelation of the residuals.
This statistic takes values in the $[0,4]$ range, it should be as close as possible to $2$, and informs about the homoscedasticity (=equality of variance) of the residuals.

## Model specification

The *response variable* `Y` is approximated as $a + b * \mathbb{1}_B + c * \mathbb{1}_C$ denoting
$a$, $b$ and $c$ the three coefficients that appear in the `coef` column:

In [ ]:
print(fitted_model.summary().tables[1])

### Design matrices

The linear model relies on a specific representation of the data: the `endog` and `exog` *design* matrices.

In [ ]:
from patsy import dmatrices
endog, exog = dmatrices('Y ~ Group', dataframe)

In [ ]:
print(statsmodels_material.side_by_side(endog, exog))

$\require{color}$

The right-hand side (`endog`) is a vector that represents the response variable we previously called *Y*.
Here, this is a vector because we model a single response variable.

The left-hand side is the (main) *design matrix* (`exog`) and represents the terms involved as input to the linear model.
As already said, fitting such a model consists in finding $a$, $b$ and $c$ such that:

$$
\mathtt{\colorbox{#F2F3F4}{Y}} = a \mbox{ } \mathtt{\colorbox{#F2F3F4}{Intercept}} + b \mbox{ } \mathtt{\colorbox{#F2F3F4}{Group[T.B]}} + c \mbox{ } \mathtt{\colorbox{#F2F3F4}{Group[T.C]}} + \epsilon
$$

As the intercept is a constant, the corresponding term is always modelled as a constant vector.

We can observe that the `Group` variable is represented as several binary variables; one per level of the original categorical variable, **minus one**.
These binary variables are called *dummy variables*. All categorical variables are translated this way, into one or several dummy variables.

`A` is not explicitly modelled, because all the values in a `Group[T.A]` column could be predicted knowing the corresponding values in the other two `Group` columns.
In other words, a `Group[T.A]` dummy variable would not bring additional information.

Basically, `A` is taken as a reference group. The intercept is enough to capture group `A`'s mean, and the `Group[T.B]` and `Group[T.C]` variables encodes the offsets with group `A`'s mean for the other 2 groups.

If we force `ols` to explicitly use an additional dummy variable for group `A`, designing the matrices ourselves, we get correct output in this case, but `ols` complains about collinearity:

In [ ]:
intercept, dummyB, dummyC = exog.T
dummyA = intercept - dummyB - dummyC
overdefined_exog = np.stack((intercept, dummyA, dummyB, dummyC), axis=1)
overdefined_exog

In [ ]:
overdefined_model = sm.OLS(endog, overdefined_exog).fit()
overdefined_model.summary()

[More about design matrices](https://en.wikipedia.org/wiki/Design_matrix).

### Wilkinson formulae

Wilkinson formulae were introduced in the S language and popularized with the advent of the R language.

In Python, this formalism is implemented by the [patsy](https://patsy.readthedocs.io/en/latest/formulas.html) package, required by statsmodels, with [minor differences](https://patsy.readthedocs.io/en/latest/R-comparison.html#r-comparison) with R.

As categorical variables may be encoded as numerical values -- in which case patsy cannot guess these variables are categorical, it is good practice to always tag these variables as categorical with the `C()` function in the formula, *e.g.* `C(Group)`.

The intercept is implicit; `Y ~ X` and `Y ~ 1 + X` are equivalent formulae. The intercept can be excluded making its contribution negative or representing it with an explicit zero: `Y ~ X - 1` or `Y ~ 0 + X`.

For example:

In [ ]:
del C # more about this later...
fitted_model = smf.ols('Y ~ C(Group) - 1', data=dataframe).fit()
print(fitted_model.summary())

<hr />

The regression coefficients are also implicit. Although `Y ~ X` corresponds to $Y = a + bX + \epsilon$, we do not write `Y ~ a + b * X`.
`a` and `b` are unknowns and are to be inferred. The terms in the formula should be column names in the dataframe.

We can introduce multiple independent variables, *i.e.* several terms, and -- optionally -- their *interactions*: `Y ~ A + B + A:B`, `Y ~ X1 + X2 + X3 + X1:X2 + X1:X3 + X2:X3 + X1:X2:X3`, etc.

`A*B` is a common short-hand for  `A + B + A:B`.

How patsy translates a formula can be checked as follows:

In [ ]:
from patsy import ModelDesc
print(ModelDesc.from_formula('Y ~ 1 + A').describe())
print(ModelDesc.from_formula('Y ~ A - 1').describe())
print(ModelDesc.from_formula('Y ~ A * B').describe())
print(ModelDesc.from_formula('Y ~ A**2').describe())

### Implementation detail

In [ ]:
%%script false --no-raise-error

C = 0 # any not-callable value, or worse: an actual function

fitted_model = smf.ols('Y ~ C(Group)', data=dataframe).fit()
# raises:

# >>> PatsyError: Error evaluating factor: TypeError: 'int' object is not callable
# >>>    Y ~ C(Group)
# >>>        ^^^^^^^^

Per default, patsy evaluates the terms in the caller namespace (here the global namespace), so that we can apply local functions to variables right in the formulae.
A common usage consists of calling `np.log`.

However, this may conflict with previously defined object names, especially in sandbox environment such as a notebook...

This default behavior can be disabled with `eval_env=-1`:

In [ ]:
fitted_model = smf.ols('Y ~ C(Group)', data=dataframe, eval_env=-1).fit()

Other issue: if a variable name is a Python reserved keyword (*e.g.* `yield`), the variable must be renamed; there is no other workarounds.

## Two-way ANOVA

Let us borrow and adapt the following data example from [statology](https://www.statology.org/two-way-anova-python/):

In [ ]:
plant_data = pd.DataFrame({'water': np.repeat(['daily', 'weekly'], 15),
                   'sun': np.tile(np.repeat(['low', 'med', 'high'], 5), 2),
                   'height': np.array([
                       6.3, 6.8, 5.5, 5.1, 6.0, 6.1, 5.0, 6.1, 3.6, 5.4,
                       6.4, 5.7, 8.3, 7.7, 7.0, 2.9, 3.2, 2.3, 3.9, 4.1,
                       3.5, 5.3, 5.8, 4.6, 3.6, 5.2, 6.2, 5.1, 6.7, 7.0,
                   ])})

In [ ]:
statsmodels_material.illustration_2way_data(plant_data)

Note: we will treat sun exposure as a *cardinal* variable and disregard the natural order of the levels.

In [ ]:
plant_model = smf.ols('height ~ water + sun', data=plant_data).fit()

Again, we can use [anova_lm](https://www.statsmodels.org/stable/generated/statsmodels.stats.anova.anova_lm.html) to print a condensed table:

In [ ]:
sm.stats.anova_lm(plant_model, typ=3) # `typ` specifies the type of sum of squares

Here, `anova_lm` prints more useful information than the omnibus statistic given by `summary`:

In [ ]:
print(plant_model.summary().tables[0])

If we look at the coefficients:

In [ ]:
plant_model.params

We can see the intercept now represents the `water=daily,sun=high` group, and -- for example -- coefficent `water[T.weekly]` encodes the difference between the intercept and the `water=weekly,sun=high` group.

Let us reconstruct the group means using the coefficients:

In [ ]:
ax = sns.swarmplot(data=plant_data, x='sun', y='height', hue='water', dodge=True)

x = ax.get_xticks()
dx = .2
w = plant_model.params

y_low_daily = w['Intercept'] + w['sun[T.low]']
y_low_weekly = w['Intercept'] + w['sun[T.low]'] + w['water[T.weekly]']
ax.plot([x[0]-dx, x[0]+dx], [y_low_daily, y_low_weekly], 'k-d', markerfacecolor='w')

y_med_daily = w['Intercept'] + w['sun[T.med]']
y_med_weekly = w['Intercept'] + w['sun[T.med]'] + w['water[T.weekly]']
ax.plot([x[1]-dx, x[1]+dx], [y_med_daily, y_med_weekly], 'k-d', markerfacecolor='w')

y_high_daily = w['Intercept']
y_high_weekly = w['Intercept'] + w['water[T.weekly]']
ax.plot([x[2]-dx, x[2]+dx], [y_high_daily, y_high_weekly], 'k-d', markerfacecolor='w');

We can appreciate the equal daily-weekly differences do not quite match the variability across the levels of the `sun` factor. As a result, the group means are not well represented, including that of the group the intercept is supposed to represent.

This inter-factor dependence is called an *interaction*.

### Interaction

To model this interaction, we need an extra term in the model:

In [ ]:
model_with_interaction = smf.ols('height ~ water * sun', data=plant_data).fit()
# remember `water * sun` is equivalent to `water + sun + water:sun`
print(sm.stats.anova_lm(model_with_interaction))

In [ ]:
model_with_interaction.params

In [ ]:
ax = sns.swarmplot(data=plant_data, x='sun', y='height', hue='water', dodge=True)

x = ax.get_xticks()
dx = .2
w = model_with_interaction.params

y_low_daily = w['Intercept'] + w['sun[T.low]']
y_low_weekly = w['Intercept'] + w['sun[T.low]'] + w['water[T.weekly]'] + w['water[T.weekly]:sun[T.low]']
ax.plot([x[0]-dx, x[0]+dx], [y_low_daily, y_low_weekly], 'k-d', markerfacecolor='w')

y_med_daily = w['Intercept'] + w['sun[T.med]']
y_med_weekly = w['Intercept'] + w['sun[T.med]'] + w['water[T.weekly]'] + w['water[T.weekly]:sun[T.med]']
ax.plot([x[1]-dx, x[1]+dx], [y_med_daily, y_med_weekly], 'k-d', markerfacecolor='w')

y_high_daily = w['Intercept']
y_high_weekly = w['Intercept'] + w['water[T.weekly]']
ax.plot([x[2]-dx, x[2]+dx], [y_high_daily, y_high_weekly], 'k-d', markerfacecolor='w');

statsmodels features an `interaction_plot` helper function, but it does not play nicely with seaborn's `swarmplot` for example and, as a result, we wrap it into another function (see the *statsmodels_material.py* file):

In [ ]:
statsmodels_material.interaction_plot(plant_data)

### Treating interaction

As we found significant interaction, we should rerun the ANOVA in the shape of one-way ANOVA, with one factor, for each level of the other factor, and possibly vice-versa.

<table><tr><td><img src="../images/two-way-anova-interaction-significant-flowchart.png" /></td></tr>
<tr><td><a href="https://www.spss-tutorials.com/spss-two-way-anova-interaction-significant/">SPSS recommendation for two-way ANOVA interaction</a></td></tr></table>

In [ ]:
daily_water_model  = smf.ols('height ~ sun',   data=plant_data[plant_data['water']=='daily']).fit()
weekly_water_model = smf.ols('height ~ sun',   data=plant_data[plant_data['water']=='weekly']).fit()
low_sun_model      = smf.ols('height ~ water', data=plant_data[plant_data['sun']=='low']).fit()
med_sun_model      = smf.ols('height ~ water', data=plant_data[plant_data['sun']=='med']).fit()
high_sun_model     = smf.ols('height ~ water', data=plant_data[plant_data['sun']=='high']).fit()

If main effects are found to be significant, we can proceed to performing post-hoc tests.

In [ ]:
daily_water_model.f_pvalue

In [ ]:
daily_water_posthoc = daily_water_model.t_test_pairwise('sun')
daily_water_posthoc.result_frame

In [ ]:
weekly_water_model.f_pvalue

In [ ]:
weekly_water_posthoc = weekly_water_model.t_test_pairwise('sun').result_frame

Problem: although `t_test_pairwise` includes a correction for multiple comparisons, this correction does not account for the multiple calls to `t_test_pairwise` we perform on the same data.

## Post-hoc tests and multiple comparisons

### The multiple comparisons problem

Let us perform 1,200 tests whom 100 should lead to a significant different. The test used features the following properties:

In [ ]:
power = 0.8
type1_error_rate = 0.05

Red pixels represent the tests (comparisons) for which $H_0$ is false (right figure) or rejected (left figure):

In [ ]:
statsmodels_material.illustration_multiple_comparisons(power, type1_error_rate)

The top right group of tests with false $H_0$ is affected by type-2 errors, or equivalently by the power of the test (`power = 1 - type2_error_rate`).

The original $5%$ significance level of the test translates into $5%$ type-1 errors that become visible when the test is applied many times as we did above.

We want the significance level to apply to the "whole picture", and to control the *family-wise error rate*. Basically, we want our $5%$ level to upper-bound the risk of erroneously rejecting any single $H_0$ (or more).

This can be done with a procedure called *correction for multiple comparisons*.

### multipletests

If we consider all 5 factored models, we may proceed to performing up to 9 comparisons, but again `t_test_pairwise` would not properly take this into account.

Note that you do not need to perform all possible comparisons. Choose what comparisons you are interested in, but do so prior to performing them.

We should use [multipletests](https://www.statsmodels.org/stable/generated/statsmodels.stats.multitest.multipletests.html) instead, for the purpose of correcting the $p$-values:

In [ ]:
from statsmodels.stats.multitest import multipletests

significance_level = 0.05

all_comparisons = []
for factor1, factor2 in (('sun', 'water'), ('water', 'sun')):
    for f2_level in np.unique(plant_data[factor2]):
        model = smf.ols(f'height ~ {factor1}', data=plant_data[plant_data[factor2]==f2_level]).fit()
        if model.f_pvalue <= significance_level:
            model_name = f'{factor2}={f2_level}'
            pairwise_tests = model.t_test_pairwise(factor1).result_frame
            pairwise_tests.index = [ f'{comparison}[{model_name}]' for comparison in pairwise_tests.index ]
            all_comparisons.append(pairwise_tests)
all_comparisons = pd.concat(all_comparisons)

len(all_comparisons)

In [ ]:
all_comparisons['reject-hs'], all_comparisons['pvalue-hs'], _, _ = multipletests(all_comparisons['P>|t|'], alpha=significance_level)
all_comparisons

In [ ]:
statsmodels_material.confidence_intervals(all_comparisons)

Beware: the confidence intervals from [t_test_pairwise](https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.OLSResults.t_test_pairwise.html)'s table are not corrected for multiple comparisons.

### Bonferroni, Šidák and Holm

[`multipletests`](https://www.statsmodels.org/stable/generated/statsmodels.stats.multitest.multipletests.html) implements several correction procedures. The Holm correction with Šidák adjustments is the default method (`holm-sidak`).

If we perform $n$ tests, the $p$-value for each test can be adjusted as follows:

* Bonferroni adjustement : $p_{corrected} = np$
* Šidák adjustment : $p_{corrected} = 1 - ( 1 - p )^n$

In the Holm's procedure, we sequentially consider each $p$-value, starting from the smallest one, and adjust them on basis of the number of remaining $p$-values to adjust.
Basically:

1. the smallest $p$-value is adjusted considering $n$ multiple comparisons (because we have not adjusted any $p$-value yet),
2. the second smallest $p$-value is adjusted considering $n-1$ multiple comparisons (because we have already adjusted one $p$-value),
3. and so on.

Compared with pingouin's [`multicomp`](https://pingouin-stats.org/build/html/generated/pingouin.multicomp.html#pingouin.multicomp), statsmodels' `multipletests` includes more False Discovery Rate (FDR)-based correction methods.

## Types of sums of squares

`anova_lm` takes an argument `typ` that can be any of `1`, `2` and `3`.

Indeed, to quantify the contribution of each term to the model, we can choose between three ways of decomposing the total variance or sum of squares.

Let us consider the following model: `Y ~ A + B + A:B`

#### Type-1

* A's contribution will evaluated comparing `Y ~ A` vs `Y ~ 1`
* B: `Y ~ A + B` vs `Y ~ A`
* A:B (interaction term): `Y ~ A + B + A:B` vs `Y ~ A + B`

#### Type-2

* A: `Y ~ A + B` vs `Y ~ B`
* B: `Y ~ A + B` vs `Y ~ A`
* A:B: `Y ~ A + B + A:B` vs `Y ~ A + B`

Type-2 is often chosen for regression problems (with continuous predictors).

#### Type-3

* A: `Y ~ A + B + A:B` vs `Y ~ B + A:B`
* B: `Y ~ A + B + A:B` vs `Y ~ A + A:B`
* A:B: `Y ~ A + B + A:B` vs `Y ~ A + B`

Type-3 is suitable for multi-factorial designs (with several categorical factors) and unbalanced groups.

## Mixed-effects models

Let us consider the reaction-time dataset available at [osf.io/asq8n](https://osf.io/asq8n):

In [ ]:
rt_data = pd.read_csv('https://osf.io/download/asq8n/')
rt_data.T

In [ ]:
model = smf.mixedlm('utterancelength ~ place * gender', rt_data, groups='subject').fit()
model.summary()

In [ ]:
model = smf.mixedlm('utterancelength ~ place * gender', rt_data, groups='subject', vc_formula={'item': '0 + item'}, re_formula='1').fit()
model.summary()

In [ ]:
#sm.stats.anova_lm(model) # does not work!

Instead of the traditional sums-of-squares and ANOVA, we test for main effects by comparing models using the Wald test.

We compare the full model with a model without the effect of interest. This second model is specified as a constraint on one or more model coefficients. For example:

In [ ]:
model.wald_test('gender[T.male] = 0', scalar=True, use_f=False)

In the above example, we have established that the `gender` factor has no effects (`p-value>0.05`).

The $p$-values reported by `summary` on coefficients associated with binary factors readily inform about the factor's effect.

Multi-level main effects can be tested with null hypothesis: *all related coefficients equal to $0$*.

In [ ]:
model.wald_test('place[T.velar] = place[T.labial] = 0', scalar=True, use_f=False)

If a factor or interaction term does not exhibit an effect, it can be removed from the model. This is often done for interaction terms:

In [ ]:
model.wald_test('place[T.labial]:gender[T.male] = place[T.velar]:gender[T.male] = 0', scalar=True, use_f=False)

Instead of posthoc tests, one can test individual pairwise differences:

In [ ]:
model.wald_test('place[T.velar] = place[T.labial]', scalar=True, use_f=False)

## Regression

What if -- instead of factors -- our independent variables are continuous variables?

### Ordinary Least Squares

In [ ]:
patients = pd.read_csv('../data/patients.csv')
patients.head()

In [ ]:
sns.scatterplot(data=patients, x='CHUK', y='Response', label='Patient');

In [ ]:
model = smf.ols('Response ~ CHUK', patients).fit()
#print(model.summary().tables[0])
print(model.summary().tables[1])

<p style="font-size: x-small;">Data set and choice of an explanatory variable inspired by the RS3 session about linear models on <a href="https://moodle01.hosting.pasteur.fr">Institut Pasteur's Moodle</a></p>

In [ ]:
statsmodels_material.illustration_regression(patients, model)

In [ ]:
model.params

Similarly to previous OLS applications, the coefficients of the model can be found in the `coef` column.

$
\texttt{Response} = a + b\mbox{ }\texttt{CHUK} + \epsilon
$

with intercept $a = -11.2792$ and slope $b = 0.9727$.

### Residual plots

To assess the adequacy of the model, we inspect the residuals $\epsilon_i$ in various.
First, we plot the residuals *vs* the explanatory variable:

In [ ]:
model.resid

In [ ]:
statsmodels_material.illustration_regression_residuals(patients, model)

The most distant points may be outliers.

We expect the residuals not to exhibit any structure:

* systematic (positive-only or negative-only) errors on subdomains of the explanatory variable are indicative of the model not being flexible enough,
* the dispersion of the residuals should not vary as a function of the explanatory variable (homoscedasticity).

<table width=60%><tr><td><img src="../images/heteroskedasticity.png" /></td></tr>
<tr><td><a href="https://towardsdatascience.com/heteroscedasticity-is-nothing-to-be-afraid-of-730dd3f7ca1f">"Heteroscedasticity is nothing to be afraid of" - Sachin Date</a></td></tr></table>

Criterion: the residuals should be normally distributed.

In [ ]:
sm.graphics.qqplot(model.resid, fit=True, line='45', fmt='b', marker='+');

Note: statsmodels's `qqplot` compares with R's qqplot as long as `fit=True` to standardize the residuals.
`fit=True` makes `qqplot` differs from scipy's `probplot`.

In what refers to normality of the residuals, the last summary table is also informative.

In [ ]:
print(model.summary().tables[-1])

The hypothesis of normality of the residuals is rejected.

### Influence plots

To identify outliers and influential points, statsmodels features more [diagnostic measures and plots](https://www.statsmodels.org/stable/generated/statsmodels.stats.outliers_influence.OLSInfluence.html):

In [ ]:
from statsmodels.stats.outliers_influence import OLSInfluence
diagnostics = OLSInfluence(model)

In [ ]:
diagnostics.summary_frame()

In [ ]:
_, axes = plt.subplots(1, 2, figsize=(13.3,4.1))

diagnostics.plot_influence(ax=axes[0])
axes[0].axhline(0, linestyle=':', linewidth=1)

diagnostics.plot_index(threshold=0.02, ax=axes[1])
axes[1].axhline(0, linestyle=':', linewidth=1);

### Leverage and Cook's distance

Let us consider a smaller data sample:

In [ ]:
np.random.seed(237598)
x = stats.lognorm.rvs(1, size=30)
y = np.log(4 + x + stats.norm.rvs(size=x.size))

In [ ]:
ax = sns.scatterplot(x=x, y=y)
ax.set_xlabel('x')
ax.set_ylabel('y');

In [ ]:
sns.regplot(x=x, y=y);

In [ ]:
X = np.stack((np.ones_like(x), x), axis=1)
model = sm.OLS(y, X).fit()
diagnostics = OLSInfluence(model)

In [ ]:
_, axes = plt.subplots(1, 2, figsize=(13.3,4.1))

diagnostics.plot_influence(ax=axes[0])
axes[0].axhline(0, linestyle=':', linewidth=1)
diagnostics.plot_index(threshold=0.02, ax=axes[1])
axes[1].axhline(0, linestyle=':', linewidth=1);

In [ ]:
high_leverage_point = np.argmax(diagnostics.hat_matrix_diag) # 20
cooks_distant_point = np.argmax(diagnostics.cooks_distance[0]) # 5

In [ ]:
statsmodels_material.illustration_outlier(x, y, high_leverage_point, cooks_distant_point)

The leverage for a point tells how much the model would change if we move the response value of that point, while Cook's distance reflects how much the model changes if we omit the point.

Therefore, Cook's distance is an “effect size” for outliers. Influential points that fall above $1$ are undesirable and should preferably be removed or trimmed (see also [robust linear models](https://www.statsmodels.org/stable/generated/statsmodels.robust.robust_linear_model.RLM.html)). A Cook's distance between $0.5$ and $1$ signals a point (=an observation) to be examined.

Note: compared to other implementations of influence plots, statsmodels' influence plot lacks the Cook's distance isocurves.

## Non-linear regression

### Data transformation

In the previous simulated data example, the relationship between the explanatory and response variables is actually not linear. The true model is:

In [ ]:
sns.regplot(x=np.log(x), y=y);

Such simple non-linear relationships are common and a useful trick consists in transforming the explanatory variables using monotonous functions.

Examples (heavily inspired by the RS3 session about linear models on [Institut Pasteur's Moodle](https://moodle01.hosting.pasteur.fr)):

In [ ]:
statsmodels_material.illustration_monotonous_functions()

Generic standardization functions exist, such as the [Box-Cox transform](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.boxcox.html) in scipy, but they often require the explanatory variable to take positive values and the interpretation of the relationship becomes less straight-forward.

### Polynomial regression

Let us consider some almost-linearly related data:

In [ ]:
def get_sample(n=100):
    # we will use this code again later in the notebook
    x = np.sort(stats.uniform.rvs(0, 2, size=n))
    y_th = 1 + 0.5 * x + 1.5 * x**2 + 0.3 * x**3
    y = y_th + 2 * stats.norm().rvs(size=x.size)
    return x, y, y_th

np.random.seed(237598)
x, y, y_th = get_sample()

df = pd.DataFrame({'x': x, 'y': y})

ax = sns.scatterplot(x='x', y='y', data=df, label='observations')

#ax.plot(x, y_th, 'r-', label='true relationship') # uncomment

ax.set_xlabel('$x$ (explanatory)')
ax.set_ylabel('$y$ (response)')
ax.legend();

A linear model performs very well, but the structured errors leave no doubt the model is not flexible enough:

In [ ]:
df = pd.DataFrame({'x': x, 'y': y})
linear_model = sm.OLS.from_formula('y ~ x', df).fit()

_, axes = plt.subplots(1, 2, figsize=(13.3,4.1))
sns.regplot(x='x', y='y', data=df, ax=axes[0], line_kws=dict(color='r', linewidth=1.5))
sns.scatterplot(x='x', y='residuals', data=df.assign(residuals=linear_model.resid), ax=axes[1])
axes[1].axhline(0, linestyle=':', linewidth=1);

A flexible approach consists of introducing powers of the explanatory variable, in the shape of multiple data columns.

$$
Y = \left[ X^0, X^1, X^2, ... \right]\beta + \epsilon
$$
or similarly $y_i = \beta_0 + \beta_1 x_i +\beta_2 x_i^2 +... +\epsilon_i$ for all observation $i$.

For example:

In [ ]:
augmented_df = df.assign(x2 = x**2)[['y', 'x', 'x2']]
augmented_df.head()

In [ ]:
poly2_model = smf.ols('y ~ 1 + x + x2', augmented_df).fit()

As the predictors we plug into the model are synthetic, we do not need to model any interaction between them.

Similarly, we can manually define the `exog` matrix:

In [ ]:
X_poly2 = np.stack((np.ones_like(x), x, x*x), axis=1)
poly2_model_bis = sm.OLS(y, X_poly2).fit()

In [ ]:
statsmodels_material.illustration_nonlinear_regression(df, y_th, poly2_model, 2)

Polynomial models are flexible enough to closely approximate any function in the neighborhood of a point (think of Taylor series expansions) but, of course, may not be adequate enough as we are modelling a function across an entire domain.

It is also possible to introduce any other non-linear transformation of the explanatory variable as additional terms in the modelling equation or columns in the design matrix. See the following examples in statsmodels documentation: [1](https://www.statsmodels.org/stable/examples/notebooks/generated/predict.html) [2](https://www.statsmodels.org/stable/examples/notebooks/generated/ols.html).

### Overfitting

Let us introduce higher order terms:

In [ ]:
augmented_again_df = augmented_df.assign(x3=x**3, x4=x**4, x5=x**5, x6=x**6) # yippee!
poly6_model = smf.ols('y ~ 1 + x + x2 + x3 + x4 + x5 + x6', augmented_again_df).fit()

In [ ]:
statsmodels_material.illustration_nonlinear_regression(df, y_th, poly6_model, 6)

If we compare the various models, we can observe that more complex models tend to perform better on the training data.

In [ ]:
scores = pd.DataFrame(np.array(
    [[model.rsquared, model.rsquared_adj, model.llf, model.aic, model.bic] \
        for model in (linear_model, poly2_model, poly6_model)]),
    index=['1', '2', '6'],
    columns=['R2', 'R2_adjusted', 'log-likelihood', 'AIC', 'BIC'])
scores

Now if we get a new sample from the same population, and compute the coefficient of determination:

In [ ]:
x_test, y_test, _ = get_sample()

In [ ]:
statsmodels_material.illustration_R2_poly(x, y, x_test, y_test)

...the over-complex models perform poorly.

### Model selection

Choosing among models should not rely on data fitness only, especially if we only consider the data used to fit the model.

To choose between models, two strategies:

* model evaluation on test data, *i.e.* a second (sub-)sample drawn from the same population as the data used to fit the model,
    * => `scikit-learn`
* heuristics; for example, model complexity is to be controlled, so that simpler models are favored over complex models.

### Information criteria

[Akaike (AIC)](https://en.wikipedia.org/wiki/Akaike_information_criterion) and [Bayesian (BIC)](https://en.wikipedia.org/wiki/Bayesian_information_criterion) information criteria combine model fitness with the notion of model complexity.

In [ ]:
scores

$$
AIC = 2k - 2\log{L}
$$
$$
BIC = k\log{n} - 2\log{L}
$$

with $\log{L}$ the maximum log-likelihood we also met in the OLS summary, and quantifies the goodness-of-fitness.

$k$ is the number of estimated parameters in the model, and $n$ the number of observations.
For $p$ predictors (explaining variables), a linear model's $k=p+2$ because we also estimate an intercept and the error variance.

The likelihood is the probability that the data are generated by the model: $L=P\left(X,y|\mathcal{M}(\theta)\right)$, denoting $\mathcal{M}(\theta)$ the model with parameters $\theta$ (estimated coefficients).

In the case of a linear regression with normally-distributed residuals: $\log{L}\propto -\frac{\sum_i(y_i - \textbf{x}_i^\top\beta)^2}{2\sigma^2}$ with $\beta$ are the regression coefficients.

`OLS` models rely on such a form for the likelihood and that is the reason why we must ensure the residuals are normally distributed.

In [ ]:
statsmodels_material.illustration_AIC_BIC_poly(x, y)

In a linear model of several factors and many potential interactions terms, the AIC is often used to compare between all the possible models and select a model that exhibits a good trade-off between the likelihood (higher is better) and the number of terms (fewer is better).

## Generalized linear models

What if -- instead of a continuous variable -- our response variable is a binary (or categorical) variable?

### Link functions and families

We need two ingredients.

First, we apply a transformation to the response variable to project its values onto $\mathbb{R}$:

$$
g(y) = \textbf{X}\beta + \epsilon
$$

$g$ is called a *link function*. 

At the same time, a *family* of distribution functions is chosen for $y|\textbf{X}$, to express $\mathbb{E}[y|\textbf{X}] = g^{-1}(\textbf{X}\beta)$. Importantly, the chosen distribution determines the relationship between the mean and variance of $y|\textbf{X}$.

### Example problems

#### Count outcome

\[copying [Wikipedia](https://en.wikipedia.org/wiki/Generalized_linear_model)\]
Suppose we want to predict how many people will come to some type of outdoor places (*e.g.* beaches) as a function of temperature.

If we observed the attendance in multiple occasions, mostly at temperatures in the 15-35°C range, a fitted linear model could predict impossible values, namely negative attendance, at low temperatures, say 5°C.

A link function could be used to turn the attendance into a variation rate of attendance, so that the attendance can be multiplied or divided as a function of temperature increase/decrease, and never subtracted.

To do so, we need an *exponential response* model with $g=\log$. If $g(y)$ varies linearly with temperature, $y$ will be Poisson distributed.

In [ ]:
family = sm.families.Poisson()
# an optional first argument specifies the link function; default is the log link function

#### Odds ratio

Suppose now we want to model the probability of a two-option process, *e.g.* «survives *vs* dies».

A response variable is always quantitative. If we choose it to be the raw probability, we may find situations such that the predicted probability takes a negative value, or falls above $1$.

If we think of the effect of a change in the value of an explanatory variable, this effect better applies to the *odds* of survival, *i.e.* the ratio of survival over death, and again we want the model to make this value vary in a multiplicative way (no subtraction, no addition).

This is a typical application of a *logit* link function, which relies on $y$ following a binomial distribution.

In [ ]:
family = sm.families.Binomial()
# the default link function is logit

### Choosing families and link functions

Common choices are:
* $y$ is continuous in $\mathbb{R}$: Gaussian distribution with identity link function (standard linear model)

In [ ]:
family = sm.families.Gaussian()
# the default link function is the identity

* $y$ is continuous in $\mathbb{R}^+$: gamma or inverse Gaussian distribution with log link function

In [ ]:
family = sm.families.Gamma(sm.families.links.Log())
family = sm.families.InverseGaussian(sm.families.links.Log())
# the log link function must be specified, as the log link function is not default

* $y$ is count data ($y \in \mathbb{N}^+$): Poisson distribution (see above)
* $y$ is $0$ or $1$: binomial distribution with *logit* link function (see above)

### Usage example

In [ ]:
df = pd.read_csv('../data/titanic.csv')
df.head()

In [ ]:
model = smf.glm('Survived ~ Age + C(Pclass) + C(Sex)', df, family=sm.families.Binomial())
model = model.fit()
model.summary()

Some general statistics are not shown in the tables above, but still avaible in the *Generalized Linear Model Regression Results*  object, such as the AIC or BIC:

In [ ]:
model.aic

Main effects and pairwise differences can be tested using Wald test as already shown.

## Repeated-measures ANOVA and sphericity

Example (one-way): each animal observed multiple times, *e.g.* at different ages; and we are not interested in the putative differences between animals.

$$
SS_{\textrm{total}} = SS_{\textrm{treatment}} + (SS_{\textrm{subject}} + SS_{\textrm{error}})
$$

$$
F^* = \frac{\frac{SS_{\textrm{treatment}}}{k - 1}}{\frac{SS_{\textrm{error}}}{(k-1)(n-1)}}
$$

Designs are balanced.

Let us borrow an example from `pingouin` documentation:

In [ ]:
import pingouin as pg

data = pg.read_dataset('rm_anova2')
data.loc[[0,1,10,11,20,21,30,31]]

In this example, each subject (`Subject`) has undergone all possible measurements, for all levels of the `Time` and `Metric` factors.
As a consequence, the observations for each subject are not independent, and this must be accounted for by the model.

In a standard repeated measures ANOVA, the covariance structure is just assumed to exhibit a property called sphericity.

`Time` and `Metric` are called *within-subject* factors.

statsmodels features [AnovaRM](https://www.statsmodels.org/stable/generated/statsmodels.stats.anova.AnovaRM.html) but corrections for departure from sphericity are not implemented and we should first perform a Mauchly's test for sphericity, for example with [pingouin.sphericity](https://pingouin-stats.org/generated/pingouin.sphericity.html):

In [ ]:
pg.sphericity(data, dv='Performance', subject='Subject', within=['Time', 'Metric'])

In [ ]:
from statsmodels.stats import anova
result = anova.AnovaRM(data, depvar='Performance', subject='Subject', within=['Time', 'Metric']).fit()
result.anova_table

In contrast, [rm_anova](https://pingouin-stats.org/generated/pingouin.rm_anova.html) from pingouin does implement Greenhouse-Geiser correction.

In [ ]:
pg.rm_anova(data, dv='Performance', subject='Subject', within=['Time', 'Metric'])

Mixed effects models are increasingly popular and preferred over the standard repeated measures ANOVA, especially because sphericity simply cannot be expected from the data in most cases.